In [0]:
from keras.models import Model, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, Average, Dropout
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import Adam
from keras.datasets import cifar10
import numpy as np

Using TensorFlow backend.


In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import sys
sys.path.append('drive/Project')

In [0]:
#!ls "/content/drive/My Drive"

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train/255
x_test = x_test/255
y_train = to_categorical(y_train, 10)


In [0]:
print("x_train shape: ",format(x_train.shape))
print("y_train shape: ",format(y_train.shape))
print("x_test shape: ",format(x_test.shape))
print("y_test shape: ",format(y_test.shape))

x_train shape:  (50000, 32, 32, 3)
y_train shape:  (50000, 10)
x_test shape:  (10000, 32, 32, 3)
y_test shape:  (10000, 1)


In [0]:
input_shape = x_train[0,:,:,:].shape
model_input = Input(shape=input_shape)

In [0]:
epochs = 5

In [0]:
#First CNN - ConvPool cnn

def conv_pool_cnn(model_input):
  x = Conv2D(96, (3,3), activation='relu', padding='same')(model_input)
  x = Conv2D(96, (3,3), activation='relu', padding='same')(x)
  x = Conv2D(96, (3,3), activation='relu', padding='same')(x)
  x = MaxPooling2D(pool_size = (3,3), strides = 2)(x)
  x = Conv2D(192, (3,3), activation='relu', padding='same')(x)
  x = Conv2D(192, (3,3), activation='relu', padding='same')(x)
  x = Conv2D(192, (3,3), activation='relu', padding='same')(x)
  x = MaxPooling2D(pool_size = (3,3), strides = 2)(x)
  x = Conv2D(192, (3,3), activation='relu', padding='same')(x)
  x = Conv2D(192, (1,1), activation='relu')(x)
  x = Conv2D(10, (1,1))(x) # No activation here, since its passing through Global Average Pooling layer
  x = GlobalAveragePooling2D()(x)
  x = Activation(activation='softmax')(x)
  
  model = Model(model_input, x, name='conv_pool_cnn')
  
  return model

In [0]:
#instantiate the model
convPoolCnnModel = conv_pool_cnn(model_input)

In [0]:
def compile_and_train(model, num_epochs):
  model.compile(loss=categorical_crossentropy, optimizer=Adam(), metrics=['acc'])
 # history = model.fit(x=x_train, y=y_train, batch_size=32, epochs=num_epochs, verbose=1, validation_split=0.2)
  
  
  filepath = model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
  
  checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_weights_only=True, save_best_only=True, mode='auto', period=1)
  
  tensor_board = TensorBoard(log_dir='logs/', histogram_freq=0, batch_size=32)
  
  history = model.fit(x=x_train, y=y_train, batch_size=32, epochs=num_epochs, verbose=1, callbacks=[checkpoint, tensor_board], validation_split=0.2)
   
 
  return history

In [0]:
_ = compile_and_train(convPoolCnnModel, num_epochs=epochs)

Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================] - 60s 1ms/step - loss: 1.8740 - acc: 0.2860 - val_loss: 1.5329 - val_acc: 0.4251
Epoch 2/5
40000/40000 [==============================] - 59s 1ms/step - loss: 1.3500 - acc: 0.5065 - val_loss: 1.2175 - val_acc: 0.5618
Epoch 3/5
40000/40000 [==============================] - 59s 1ms/step - loss: 1.0640 - acc: 0.6182 - val_loss: 0.9703 - val_acc: 0.6568
Epoch 4/5
40000/40000 [==============================] - 59s 1ms/step - loss: 0.8766 - acc: 0.6885 - val_loss: 0.8284 - val_acc: 0.7102
Epoch 5/5
40000/40000 [==============================] - 59s 1ms/step - loss: 0.7481 - acc: 0.7388 - val_loss: 0.7455 - val_acc: 0.7433


In [0]:
#evaluate the model

def evaluate_error(model):
  pred = model.predict(x_test, batch_size=32)
  pred = np.argmax(pred, axis=1)
  pred = np.expand_dims(pred, axis=1)
  error = np.sum(np.not_equal(pred,y_test))/y_test.shape[0]
  
  return error



In [0]:
evaluate_error(convPoolCnnModel)

0.258

In [0]:
#Second Model

def all_cnn(model_input):
  x = Conv2D(96, (3,3), activation='relu', padding='same')(model_input)
  x = Conv2D(96, (3,3), activation='relu', padding='same')(x)
  x = Conv2D(96, (3,3), activation='relu', padding='same', strides=2)(x)
  x = Conv2D(192, (3,3), activation='relu', padding='same')(x)
  x = Conv2D(192, (3,3), activation='relu', padding='same')(x)
  x = Conv2D(192, (3,3), activation='relu', padding='same', strides=2)(x)
  x = Conv2D(192, (3,3), activation='relu', padding='same')(x)
  x = Conv2D(192, (1,1), activation='relu')(x)
  x = Conv2D(10, (1,1))(x) # No activation here, since its passing through Global Average Pooling layer
  x = GlobalAveragePooling2D()(x)
  x = Activation(activation='softmax')(x)
  
  model = Model(model_input, x, name='all_cnn')
  
  return model


In [0]:
allCnnModel = all_cnn(model_input)
_ = compile_and_train(allCnnModel, num_epochs=epochs)

Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================] - 56s 1ms/step - loss: 1.9413 - acc: 0.2550 - val_loss: 1.6226 - val_acc: 0.3939
Epoch 2/5
40000/40000 [==============================] - 56s 1ms/step - loss: 1.5408 - acc: 0.4304 - val_loss: 1.4378 - val_acc: 0.4713
Epoch 3/5
40000/40000 [==============================] - 55s 1ms/step - loss: 1.3455 - acc: 0.5071 - val_loss: 1.2998 - val_acc: 0.5264
Epoch 4/5
40000/40000 [==============================] - 55s 1ms/step - loss: 1.1569 - acc: 0.5834 - val_loss: 1.0810 - val_acc: 0.6114
Epoch 5/5
40000/40000 [==============================] - 55s 1ms/step - loss: 1.0091 - acc: 0.6412 - val_loss: 0.9967 - val_acc: 0.6465


In [0]:
evaluate_error(allCnnModel)

0.3541

In [0]:
#3rd model Network-in-Network model 

def nin_cnn(model_input):
  
  #mlp conv block1
  x = Conv2D(32, (5,5), activation='relu', padding='valid')(model_input)
  x = Conv2D(32, (1,1), activation='relu')(x)
  x = Conv2D(32, (1,1), activation='relu')(x)
  x = MaxPooling2D((2,2))(x)
  x = Dropout(0.5)(x)
  
  #mlp conv block2
  x = Conv2D(64, (3,3), activation='relu', padding='valid')(x)
  x = Conv2D(64, (1,1), activation='relu')(x)
  x = Conv2D(64, (1,1), activation='relu')(x)
  x = MaxPooling2D((2,2))(x)
  x = Dropout(0.5)(x)
  
  #mlp conv block3
  x = Conv2D(128, (3,3), activation='relu', padding='valid')(x)
  x = Conv2D(32, (1,1), activation='relu')(x)
  x = Conv2D(10, (1,1))(x)
  
  x = GlobalAveragePooling2D()(x)
  x = Activation(activation='softmax')(x)
  
  model = Model(model_input, x, name='nin_cnn')
  
  return model 

In [0]:
ninModel = nin_cnn(model_input)
_ = compile_and_train(ninModel, num_epochs=epochs)

Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================] - 22s 546us/step - loss: 1.9284 - acc: 0.2661 - val_loss: 1.6451 - val_acc: 0.3824
Epoch 2/5
40000/40000 [==============================] - 23s 566us/step - loss: 1.5890 - acc: 0.4140 - val_loss: 1.4451 - val_acc: 0.4733
Epoch 3/5
40000/40000 [==============================] - 23s 563us/step - loss: 1.4564 - acc: 0.4665 - val_loss: 1.3365 - val_acc: 0.5184
Epoch 4/5
40000/40000 [==============================] - 23s 564us/step - loss: 1.3708 - acc: 0.5017 - val_loss: 1.2752 - val_acc: 0.5390
Epoch 5/5
40000/40000 [==============================] - 23s 566us/step - loss: 1.3044 - acc: 0.5295 - val_loss: 1.2793 - val_acc: 0.5430


In [0]:
evaluate_error(ninModel)

0.4584

In [0]:
conv_pool_cnn_model = conv_pool_cnn(model_input)
all_cnn_model = all_cnn(model_input)
nin_cnn_model = nin_cnn(model_input)

conv_pool_cnn_model.load_weights('conv_pool_cnn.05-0.75.hdf5')
all_cnn_model.load_weights('all_cnn.05-1.01.hdf5')
nin_cnn_model.load_weights('nin_cnn.05-1.30.hdf5')

models = [conv_pool_cnn_model, all_cnn_model, nin_cnn_model]

In [0]:
def ensemble(models, model_input):
    
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    
    model = Model(model_input, y, name='ensemble')
    
    return model
ensemble_model = ensemble(models, model_input)

In [0]:
evaluate_error(ensemble_model)

0.2677

In [0]:
pair_A = [conv_pool_cnn_model, all_cnn_model]
pair_B = [conv_pool_cnn_model, nin_cnn_model]
pair_C = [all_cnn_model, nin_cnn_model]
pair_A_ensemble_model = ensemble(pair_A, model_input)
evaluate_error(pair_A_ensemble_model)

0.3047